In [1]:
import xarray as xr
from glob import glob
from os.path import join

In [2]:
day = "out_2012-06-05T00Z/*"

rollout = "/glade/derecho/scratch/dkimpara/CREDIT_runs/fcnn_6_12h_train_filter/forecast/"
rollout = "/glade/derecho/scratch/dkimpara/CREDIT_runs/camulator_weather_rollout/forecast/2012-06-05T00Z/*"
rollout = "/glade/derecho/scratch/dkimpara/CREDIT_runs/fcnn_6_12h/forecast/out_2012-06-05T00Z/*"
rollout_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_fcnn_scaled_train/forecast_backup"

all_files = sorted(glob(join(rollout_dir, day)))


In [ ]:
datasets = []
for file in all_files:
    datasets.append(xr.open_dataset(file))
DS_all = xr.concat(datasets, dim='time')

In [8]:
DS = DS_all.sel(ensemble_member_label=0)

In [9]:
batch_size = 10
num_cpus = 10
variable, level = "U", 24

In [10]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import numpy as np
import cmocean
from tqdm import tqdm
import multiprocessing as mp
import os

# Parameters for customization
central_latitude = 23.5  # Earth's axial tilt ~23.5°
fps = 12
output_file = f"Robinson_{variable}{level if variable in 'UVTQtot' else ''}_dynamic_earth.mp4"

# Assuming `DS` is a dataset object with variables 'longitude', 'latitude', 'PRECT', and 'time'
print('starting')

# Data grid
lons = DS['longitude'].values
lats = DS['latitude'].values
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Wrap longitude grid for smooth transitions
lon_grid_wrapped = np.concatenate((lon_grid, lon_grid[:, :1] + 360), axis=1)
lat_grid_wrapped = np.concatenate((lat_grid, lat_grid[:, :1]), axis=1)  # Adjust lat_grid for consistency
if variable in "UVTQtot":
    data_np = DS[variable].sel(level=level).values
else:
    data_np = DS['U10'].values

# Wrap data for smooth transitions
data_wrapped = np.concatenate((data_np, data_np[:, :, :1]), axis=2)

# Function to generate a single frame
def generate_frame(frame):
    fig = plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
    ax.set_global()
    ax.coastlines()

    vma = 20
    data = data_wrapped[frame, :, :]

    # Plot the data
    img = ax.contourf(
        lon_grid_wrapped, lat_grid_wrapped, data, 60, transform=ccrs.PlateCarree(), cmap=cmocean.cm.curl, vmin=0, vmax=vma
    )

    # Add a title based on time
    current_time = str(DS['time'].values[frame])[:10]  # Truncate to the day
    ax.set_title(f"Time: {current_time}", fontsize=14)

    # Save the frame
    frame_file = f"frame_{frame:06d}.png"
    plt.savefig(frame_file, dpi=100, bbox_inches='tight')
    plt.close(fig)
    return frame_file

# Parallel frame generation in batches
frames = np.arange(0, len(all_files), 1)
frame_files = []

for batch_start in range(0, len(frames), batch_size):
    batch_frames = frames[batch_start:batch_start + batch_size]
    with tqdm(total=len(batch_frames), desc=f"Rendering batch {batch_start // batch_size + 1}") as pbar:
        with mp.Pool(processes=num_cpus) as pool:
            for frame_file in pool.imap_unordered(generate_frame, batch_frames):
                frame_files.append(frame_file)
                pbar.update(1)

# Debugging: Check if all frames are generated
for frame_file in frame_files:
    if not os.path.exists(frame_file):
        print(f"Warning: Frame file {frame_file} does not exist.")

# Combine frames into video using ffmpeg
print("Combining frames into video...")
if frame_files:
    # os.system("ls frame_*.png")  # List generated files for debugging
    os.system(f"ffmpeg -r {fps} -pattern_type glob -i 'frame_*.png' -vf 'scale=trunc(iw/2)*2:trunc(ih/2)*2' -vcodec libx264 -pix_fmt yuv420p -y {output_file}")
else:
    print("No frames found to combine.")

print("Animation saved as", output_file)

starting


Rendering batch 1:   0%|          | 0/10 [00:00<?, ?it/s]

Rendering batch 6: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


Combining frames into video...


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopen

Animation saved as Robinson_U24_dynamic_earth.mp4


[out#0/mp4 @ 0x55b33e727440] video:1177KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.133371%
frame=   60 fps=0.0 q=-1.0 Lsize=    1179KiB time=00:00:04.83 bitrate=1998.2kbits/s speed=14.7x    
[libx264 @ 0x55b33e728b00] frame I:1     Avg QP:22.20  size: 39023
[libx264 @ 0x55b33e728b00] frame P:15    Avg QP:25.32  size: 30251
[libx264 @ 0x55b33e728b00] frame B:44    Avg QP:27.08  size: 16186
[libx264 @ 0x55b33e728b00] consecutive B-frames:  1.7%  0.0%  5.0% 93.3%
[libx264 @ 0x55b33e728b00] mb I  I16..4: 15.3% 35.8% 48.9%
[libx264 @ 0x55b33e728b00] mb P  I16..4:  1.1% 11.3% 19.3%  P16..4: 11.5% 13.0% 12.4%  0.0%  0.0%    skip:31.3%
[libx264 @ 0x55b33e728b00] mb B  I16..4:  0.4%  1.5%  3.4%  B16..8: 16.4% 17.4% 15.4%  direct:10.1%  skip:35.4%  L0:40.5% L1:37.3% BI:22.1%
[libx264 @ 0x55b33e728b00] 8x8 transform intra:33.5% inter:48.2%
[libx264 @ 0x55b33e728b00] coded y,uvDC,uvAC intra: 76.6% 86.1% 84.6% inter: 37.6% 32.6% 19.1%
[libx264 @ 0x55b33e72